In [67]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import string

In [129]:
def tagify(key, value):
    return "<" + key + ":" + value + ">"

def process_story(soup):
    processed_story = ""
    attribute_error_counts = 0
    
    processed_story += tagify(
        "EPISODE NUMBER",
        soup.find(attrs={"class": "radio-episode-num"}).getText()[:-1],
        )
    processed_story += " "
    
    processed_story += tagify(
        "EPISODE NAME",
        soup.find(attrs={"class": "radio"}).find("a").getText(),
        )
    processed_story += " "

    story = soup.findAll(attrs={"class": "act-inner"})
    for section in story:
        for speaker in section.children:
            try:
                processed_story += tagify(
                    dict(speaker.attrs)["class"][0].upper(), 
                    speaker.find("h4").getText().upper(),
                )
                processed_story += " "
                for p in speaker.findAll("p"):
                    processed_story += p.getText()
                    processed_story += " "
            except AttributeError as e:
                pass
                attribute_error_counts += 1
    
    return processed_story

In [165]:
BASE_URL = 'http://www.thisamericanlife.org/radio-archives/episode/EPISODE_NUMBER/transcript'
DOWNLOAD_EPISODES = [496, 469, 454, 450, 441, 414, 411, 312, 250]
tal_text = ""
for episode in DOWNLOAD_EPISODES:
    page_url = BASE_URL.replace("EPISODE_NUMBER", str(episode))
    page = urlopen(page_url)
    soup = BeautifulSoup(page)
    tal_text += process_story(soup)
len(tal_text.split()) * 20

In [166]:
with open("tal_text.txt", "w") as tal:
    tal.write(tal_text)